# CNNによるスリザーリンクパズル学習

In [ ]:
import numpy as np
import tensorflow as tf
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten, Conv2D, MaxPooling2D
from keras import backend as K
from keras.utils import np_utils

In [ ]:
npz = np.load("./dataset.npz")
data_x,data_y =(npz['arr_0'],npz['arr_1'])

In [ ]:
def onehot_x(value):
    return np.array([int(str(i) == value) for i in range(4)])
dataset_x = [] #onehotに変換されたdatax
dataset_y = [] #多ラベル変換されたdatay
for i in range(data_x.shape[0]):
    dx = data_x[i]
    dy = data_y[i]
    xh,xw = dx.shape
    onehots_x = []
    labels_y = []
    for j in range(xh):
        xlst = []
        ylst = []
        for k in range(xw):
            x = onehot_x(dx[j,k])
            xlst.append(x)
            ytop = dy[1,j,k]
            ybottom = dy[1,j+1,k]
            yleft = dy[0,j,k]
            yright = dy[0,j,k+1]
            ylst.append([ytop,ybottom,yleft,yright]) # top,bottom,left,right
        onehots_x.append(xlst)
        labels_y.append(ylst)
    dataset_x.append(np.array(onehots_x))
    dataset_y.append(np.array(labels_y))
    
dataset_x = np.array(dataset_x)
dataset_y = np.array(dataset_y)
print(data_x[0][0][0])
print(dataset_x[0][0][0])
print(dataset_y[0][0][0]) #data_xとdata_yはdata_xが空白なら一致しない可能性があるので注意(当たり前だけど...)

## 焦点用チャンネルを作る
最適かどうかはわからない

In [ ]:
print(dataset_x.shape)
print(dataset_y.shape)
focused_dataset_x = []
focused_dataset_y = []

for c in range(dataset_x.shape[0]):
    padx = np.zeros((dataset_x.shape[1]+4,dataset_x.shape[2]+4,4))
    padx[2:-2,2:-2,:] = dataset_x[c,:,:,:]
    for i in range(dataset_x.shape[1]):
        for j in range(dataset_x.shape[2]):
            newx = np.zeros((5,5,4))
            newx[:,:,:] = padx[i:i+5,j:j+5] #2 + i-2:2+i+3 = i :i+5
            #newx[2,2,4] = 1
            focused_dataset_x.append(newx)
            focused_dataset_y.append(dataset_y[c,i,j,:])
#for i in range(5):
    #print(focused_dataset_x[2][:,:,i])
focused_dataset_x = np.array(focused_dataset_x,dtype='float32')
focused_dataset_y = np.array(focused_dataset_y,dtype='float32')
print(focused_dataset_y[0])
print(focused_dataset_y[2])

In [ ]:
train_x = focused_dataset_x[:-5000]
valid_x = focused_dataset_x[-5000:]
train_y = focused_dataset_y[:-5000]
valid_y = focused_dataset_y[-5000:]
train_ds = tf.data.Dataset.from_tensor_slices((train_x, train_y)).shuffle(10000).batch(8)
valid_ds = tf.data.Dataset.from_tensor_slices((valid_x, valid_y)).batch(8)

In [ ]:
train_x.dtype

# モデル作成

In [ ]:
# モデルの宣言
model = Sequential()
model.add(Conv2D(64, kernel_size=(3, 3),
                 activation='relu',
                 padding='same',
                 input_shape=(5,5,4)))
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Conv2D(128, (3, 3), activation='relu', padding='same'))
model.add(MaxPooling2D((2, 2)))
model.add(Dropout(0.25))
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(512, activation='relu'))
model.add(Dense(4, activation='sigmoid'))

In [ ]:
def total_acc(y_true, y_pred):
    pred = K.cast(K.greater_equal(y_pred, 0.5), "float")
    flag = K.cast(K.equal(y_true, pred), "float")
    return K.prod(flag, axis=-1)
def binary_acc(y_true, y_pred):
    pred = K.cast(K.greater_equal(y_pred, 0.5), "float")
    flag = K.cast(K.equal(y_true, pred), "float")
    return K.mean(flag, axis=-1)
model.compile(loss='binary_crossentropy',
              optimizer='sgd',
              metrics=[total_acc,binary_acc])
history = model.fit(train_ds, epochs=300,validation_data=valid_ds)